## Note:

1. we dont handle out of vocabulary problem
2. we split each single chinese word and punctuation, and keep english words and number stick together

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
import pickle
import math
import matplotlib.pyplot as plt

In [2]:
# with open('./data/train_1_update.txt','r', encoding='utf8') as f:
with open('./data/train_1_update.txt','r', encoding='utf8') as f:
    file_text = f.read().encode('utf-8').decode('utf-8-sig')
datas = file_text.split('\n\n--------------------\n\n')[:-1]
with open('./data/train_1_pos_ner.pkl', 'rb') as f:
    pos_ner = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './data/train_1_update.txt'

In [1]:
with open('SampleData_deid.txt','r', encoding='utf8') as f:
    file_text = f.read().encode('utf-8').decode('utf-8-sig')
datas = file_text.split('\n\n--------------------\n\n')[:-1]
with open('./data/SampleData_pos_ner.pkl', 'rb') as f:
    pos_ner = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './data/SampleData_pos_ner.pkl'

In [ ]:
with open('./data/development_1.txt','r', encoding='utf8') as f:
    file_text = f.read().encode('utf-8').decode('utf-8-sig')
datas = [i.split('\n')[1] for i in file_text.split('\n\n--------------------\n\n')[:-1]]
with open('./data/development_1_pos_ner.pkl', 'rb') as f:
    pos_ner = pickle.load(f)

## origin data

In [ ]:
def parse_data_by_word_split(data, with_label=True):
    all_lines = []
    label_count = {}

    label_transform = {
        '醫師：': 'DOC',
        '個管師：': 'CAS',
        '民眾：': 'PAT'
    }

    for data in datas:
        article, *entity = data.split('\n')

        entity_parse = [[int(col) if i != 4 and i != 3 else col for i, col in enumerate(e.split('\t'))] for e in entity[1:]]
        split_article = re.sub(r"((醫師：)|(民眾：)|(個管師：)|[^A-Za-z\d ']|[A-Za-z\d\-#%&\./+×]+)", r'\1 ', article).split()
        
        if with_label:
            split_label = []
            char_idx = 0
            entity_idx = 0

            for word in split_article:
                full_label = label_transform[word] if word in label_transform else 'O'
                if entity_idx < len(entity_parse):
                    _, stidx, edidx, _, label = entity_parse[entity_idx]
                    if stidx <= char_idx < edidx:
                        prefix = 'B' if char_idx == stidx else 'I'
                        full_label = f'{prefix}-{label}'
                        if char_idx == edidx - len(word):
                            entity_idx += 1

                split_label.append(full_label)    
                char_idx += len(word)
                
            ###
            for k in split_label:
                if k not in label_count:
                    label_count[k] = 0
                label_count[k] += 1
            ###
                
        else:
            split_label = ['O' for _ in range(len(split_article))]

        all_lines.append(f'{" ".join(split_article)}\t{" ".join(split_label)}')
        
    #with open('./train_data_mod.txt', 'w', encoding='utf8') as f:
    #    f.write('\n'.join(all_lines))
    
    return all_lines


def parse_data_by_char(data, with_label=True):
    all_lines = []
    
    for data in datas:
        article, *entity = data.split('\n')

        entity_parse = [[int(col) if i != 4 and i != 3 else col for i, col in enumerate(e.split('\t'))] for e in entity[1:]]
        split_article = list(article)
        split_label = ['O' for _ in range(len(split_article))]
        
        if with_label:
            for _, stidx, edidx, _, label in entity_parse:
                for idx in range(stidx, edidx):
                    prefix = 'B' if idx == stidx else 'I'
                    split_label[idx] = f'{prefix}-{label}'
        all_lines.append(f'{" ".join(split_article)}\t{" ".join(split_label)}')
        
    return all_lines


def parse_data_by_sentence_with_neighbor(data, with_label=True, sep_limit=1, sep_chars={'，', '。'}, neighbor_len=100):
    all_lines = []
    
    for aid, data in enumerate(datas):
        article, *entity = data.split('\n')
        
        # save all entity with idx
        entity_idxs = {}
        if with_label:
            for e in entity[1:]:
                _, stidx, edidx, _, tp = e.split('\t')
                for i in range(int(stidx), int(edidx)):
                    prefix = 'B' if i == int(stidx) else 'I'
                    full_label = f'{prefix}-{tp}'
                    entity_idxs[i] = full_label
                    
        # iterate the article character 
        sid = 0
        sep_count = 0
        curr_text = []
        curr_label = []
        curr_neighbor = []

        for idx, char in enumerate(article):
            curr_text.append(char)
            curr_label.append(entity_idxs[idx] if idx in entity_idxs else 'O')
            
            if char in sep_chars:
                sep_count += 1
            
            ### one sentence ends ###
            if (char in sep_chars and sep_count >= sep_limit) or idx == len(article)-1:
                ### get post neighbor for this sentence ###
                if idx != len(article)-1:
                    curr_neighbor.append(article[idx+1:idx+1+(neighbor_len*2)])
                
                ### find substring of pre and post neighbor ###
                if len(curr_neighbor) == 2:
                    # for middle sentences
                    pre_neighbor = curr_neighbor[0][-neighbor_len//2:]
                    post_neighbor = curr_neighbor[1][:neighbor_len//2]
                elif idx == len(article)-1:
                    # for last sentence
                    #pre_neighbor = curr_neighbor[0][-neighbor_len//2:]
                    #post_neighbor = curr_neighbor[0][-neighbor_len:-neighbor_len//2]
                    pre_neighbor = curr_neighbor[0][-neighbor_len:]
                    post_neighbor = ''
                else:
                    # for first sentence
                    #pre_neighbor = curr_neighbor[0][neighbor_len//2:neighbor_len]
                    #post_neighbor = curr_neighbor[0][:neighbor_len//2]
                    pre_neighbor = ''
                    post_neighbor = curr_neighbor[0][:neighbor_len]
                
                ### record line result ###
                all_lines.append(f'{" ".join(curr_text)}\t{" ".join(curr_label)}\t{" ".join(pre_neighbor)}\t{" ".join(post_neighbor)}\t{aid}\t{sid}')
                
                ### re-init ###
                curr_text = []
                curr_label = []
                curr_neighbor = []
                sep_count = 0
                sid += 1
                
                ### get pre neighbor for next sentence ###
                if idx != len(article)-1:
                    stidx = idx-(neighbor_len*2)+1
                    stidx = 0 if stidx < 0 else stidx
                    curr_neighbor.append(article[stidx:idx+1])
    return all_lines


def parse_data_by_sentence(data, with_label=True, sep_limit=3, sep_chars={'，', '。'}):
    all_lines = []

    for aid, data in enumerate(datas):
        # get curr article and entity
        article, *entity = data.split('\n')
        cpos = pos_ner['pos'][aid]
        cner = pos_ner['ner'][aid]

        # save all entity with idx
        entity_idxs = {}
        if with_label:
            for e in entity[1:]:
                _, stidx, edidx, _, tp = e.split('\t')
                for i in range(int(stidx), int(edidx)):
                    prefix = 'B' if i == int(stidx) else 'I'
                    full_label = f'{prefix}-{tp}'
                    entity_idxs[i] = full_label

        # iterate the article character 
        sid = 0
        sep_count = 0
        curr_text = []
        curr_label = []
        curr_pos = []
        curr_ner = []

        for idx, char in enumerate(article):
            curr_text.append(char)
            curr_label.append(entity_idxs[idx] if idx in entity_idxs else 'O')
            curr_pos.append(cpos[idx] if idx in cpos else 'O')
            curr_ner.append(cner[idx] if idx in cner else 'O')
            
            if char in sep_chars:
                sep_count += 1
            
            if (char in sep_chars and sep_count >= sep_limit) or idx == len(article)-1:
                all_lines.append(f'{" ".join(curr_text)}\t{" ".join(curr_label)}\t{" ".join(curr_pos)}\t{" ".join(curr_ner)}\t{aid}\t{sid}')
                curr_text = []
                curr_label = []
                curr_pos = []
                curr_ner = []
                sep_count = 0
                sid += 1
    
    return all_lines   


def parse_data_by_sentence_into_bio(data, with_label=True, sep_limit=3, sep_chars={'，', '。'}):
    all_lines = []
    
    for aid, data in enumerate(datas):
        # get curr article and entity
        article, *entity = data.split('\n')

        # save all entity with idx
        entity_idxs = {}
        if with_label:
            for e in entity[1:]:
                _, stidx, edidx, _, tp = e.split('\t')
                for i in range(int(stidx), int(edidx)):
                    prefix = 'B' if i == int(stidx) else 'I'
                    full_label = f'{prefix}-{tp}'
                    entity_idxs[i] = full_label

        # iterate the article character 
        sid = 0
        sep_count = 0
        curr_line = []

        for idx, char in enumerate(article):
            curr_line.append(f'{char} {entity_idxs[idx] if idx in entity_idxs else "O"} {aid}')
            
            if char in sep_chars:
                sep_count += 1
            
            if (char in sep_chars and sep_count > sep_limit) or idx == len(article)-1:
                all_lines.append('\n'.join(curr_line))
                curr_line = []
                sep_count = 0
                sid += 1
    
    return all_lines


def parse_data_by_sentence_into_bio_with_max_len(data, with_label=True, max_len=400):
    all_lines = []

    for aid, data in enumerate(datas):
        # get curr article and entity
        article, *entity = data.split('\n')

        # save all entity with idx
        entity_idxs = {}
        if with_label:
            for e in entity[1:]:
                _, stidx, edidx, _, tp = e.split('\t')
                for i in range(int(stidx), int(edidx)):
                    prefix = 'B' if i == int(stidx) else 'I'
                    full_label = f'{prefix}-{tp}'
                    entity_idxs[i] = full_label

        # find all breakpoints
        np_article = np.array(list(article))
        is_split, = np.where(((np_article == '，') + (np_article == '。') + (np_article == '？') + (np_article == '！') + (np_article == ',')) == True)        
        #is_split += 1
        
        # iterate the article character 
        word_count = 0
        total_count = 0
        curr_line = []

        for idx, char in enumerate(article):
            curr_line.append(f'{char} {entity_idxs[idx] if idx in entity_idxs else "O"} {aid}')
            word_count += 1
            
            # if article ends
            if idx == len(article)-1:
                all_lines.append('\n'.join(curr_line))
                curr_line = []
                word_count = 0
            
            # if is breakpoint
            elif len(is_split) and idx == is_split[0]:
                nxt_sentence_len = (is_split[1] if len(is_split) > 1 else len(article)) - is_split[0]
                if word_count + nxt_sentence_len > max_len:
                    all_lines.append('\n'.join(curr_line))
                    curr_line = []
                    word_count = 0
                is_split = is_split[1:]
        #break
        
    return all_lines


def parse_data_by_sentence_with_max_len(data, with_label=True, max_len=400):
    all_lines = []

    for aid, data in enumerate(datas):
        # get curr article and entity
        article, *entity = data.split('\n')
        cpos = pos_ner['pos'][aid]
        cner = pos_ner['ner'][aid]

        # save all entity with idx
        entity_idxs = {}
        if with_label:
            for e in entity[1:]:
                _, stidx, edidx, _, tp = e.split('\t')
                for i in range(int(stidx), int(edidx)):
                    prefix = 'B' if i == int(stidx) else 'I'
                    full_label = f'{prefix}-{tp}'
                    entity_idxs[i] = full_label

        # find all breakpoints
        np_article = np.array(list(article))
        is_split, = np.where(((np_article == '，') + (np_article == '。') + (np_article == '？') + (np_article == '！') + (np_article == ',')) == True)
        
        # iterate the article character 
        word_count = 0
        sid = 0
        curr_text = []
        curr_label = []
        curr_pos = []
        curr_ner = []

        for idx, char in enumerate(article):
            curr_text.append(char)
            curr_label.append(entity_idxs[idx] if idx in entity_idxs else 'O')
            curr_pos.append(cpos[idx] if idx in cpos else 'O')
            curr_ner.append(cner[idx] if idx in cner else 'O')
            word_count += 1
            
            # if article ends
            if idx == len(article)-1:
                all_lines.append(f'{" ".join(curr_text)}\t{" ".join(curr_label)}\t{" ".join(curr_pos)}\t{" ".join(curr_ner)}\t{aid}\t{sid}')
                curr_text = []
                curr_label = []
                curr_pos = []
                curr_ner = []
                word_count = 0
                sid += 1
            
            # if is breakpoint
            elif len(is_split) and idx == is_split[0]:
                nxt_sentence_len = (is_split[1] if len(is_split) > 1 else len(article)) - is_split[0]
                if word_count + nxt_sentence_len > max_len:
                    all_lines.append(f'{" ".join(curr_text)}\t{" ".join(curr_label)}\t{" ".join(curr_pos)}\t{" ".join(curr_ner)}\t{aid}\t{sid}')
                    curr_text = []
                    curr_label = []
                    curr_pos = []
                    curr_ner = []
                    word_count = 0
                    sid += 1
                is_split = is_split[1:]
        #break
        
    return all_lines

## split data with sentence + neighbor data

In [ ]:
neighbor_len = 20
all_lines = parse_data_by_sentence_with_neighbor(data, with_label=True, sep_limit=1, 
                                                 sep_chars={'，', '。', '？', '?', '!', '！'}, neighbor_len=neighbor_len)

In [ ]:
with open(f'../bert-sequence-tagging/TEA/test_data_sen_sep1_neigh{neighbor_len}.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines))

## split data with max len sentence split

In [ ]:
all_lines = parse_data_by_sentence_with_max_len(data, with_label=True, max_len=125)

In [ ]:
with open(f'../bert-sequence-tagging/TEA/test_deid_sep_maxlen_125.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines))

## split data into bio with max len sentence split

In [ ]:
all_lines = parse_data_by_sentence_into_bio_with_max_len(data, with_label=True, max_len=140)

In [ ]:
with open(f'../mrc-for-flat-nested-ner/data_preprocess/example/dev_deid_sep_maxlen_140.bio', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(all_lines) + '\n\n')

## split data into bio

In [ ]:
all_lines = parse_data_by_sentence_into_bio(data, with_label=True, sep_limit=3)

In [ ]:
with open(f'../mrc-for-flat-nested-ner/data_preprocess/example/train_deid_sep_3.bio', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(all_lines) + '\n\n')

## split data with sentence

In [ ]:
all_lines = parse_data_by_sentence(datas, with_label=True, sep_limit=1, sep_chars={'。', '？', '?', '!', '！', '，'})

In [ ]:
#with open(f'../bert-sequence-tagging/TEA/train_data_sen_sep_5_ext.txt', 'w', encoding='utf8') as f:
#    f.write('\n'.join(all_lines))
    
with open(f'../bert-sequence-tagging/TEA/dev_data_sen_sep_1_ext.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines))

In [ ]:
len_count = [len(i.split('\t')[0].split()) for i in all_lines]
mean = np.mean(len_count)
std = np.std(len_count)
max_ = np.max(len_count)
min_ = np.min(len_count)

x = [i+1 for i in range(min_, max_+1)]
y = [0 for i in range(min_, max_+1)]
for c in len_count:
    y[c-min_] += 1
    
plt.plot(x, y)
plt.xlabel('length')
plt.title(f'{mean:.4f} / {std:.4f} / {max_} / {min_}')
plt.show()

In [ ]:
for idx, ele in enumerate(len_count):
    if ele > 128:
        print(ele, all_lines[idx].split('\t')[0])

## split data with max length

In [ ]:
# for training data
#all_lines = parse_data_by_word_split(datas, with_label=True)
#all_lines = parse_data_by_char(datas, with_label=True)

# for testing data
#all_lines = parse_data_by_word_split(datas, with_label=False)
all_lines = parse_data_by_char(datas, with_label=False)

all_sublines = []
max_seq_len = 510 #127 #511 #255

for article_id, line in enumerate(all_lines):
    curr_segment = []
    article_list, label_list = [_.split() for _ in line.split('\t')]
    #max_segment = int(len(label_list)/max_seq_len) + 1
    max_segment = math.ceil(len(label_list)/max_seq_len)
    for s in range(max_segment):
        curr_a = article_list[s*max_seq_len:(s+1)*max_seq_len]
        curr_l = label_list[s*max_seq_len:(s+1)*max_seq_len]
        curr_segment.append(f'{" ".join(curr_a)}\t{" ".join(curr_l)}\t{article_id}\t{s}')
    all_sublines.append('\n'.join(curr_segment))
    
#with open(f'../bert-sequence-tagging/TEA/train_data_char_sub_{max_seq_len}.txt', 'w', encoding='utf8') as f:
#    f.write('\n'.join(all_sublines))
    
with open(f'../bert-sequence-tagging/TEA/dev_data_char_sub_{max_seq_len}.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_sublines))

In [ ]:
all_lines[0].split('\t')[0].split('。')

## split data with conversation

In [ ]:
all_conversation = []

for line in all_lines:
    curr_conversation = []
    char_idx = 0
    
    article_list, label_list = [_.split() for _ in line.split('\t')]
    np_label_list = np.array(label_list)
    is_speaker, = np.where(((np_label_list == 'DOC') + (np_label_list == 'PAT') + (np_label_list == 'CAS')) == True)
    
    for idx in range(len(is_speaker)-1):
        stidx, edidx = is_speaker[idx], is_speaker[idx+1]
        curr_conversation.append((article_list[stidx:edidx], label_list[stidx:edidx]))
    
    all_conversation.append(curr_conversation)
    
with open('./train_data_conversation.pkl', 'wb') as f:
    pickle.dump(all_conversation, f)

## find max conversation in one training data

In [ ]:
all_lines_short = []
max_seq_len = 128

word_count = 0
curr_a = []
curr_l = []

for aid, curr_conversation in enumerate(all_conversation):
    for cid, (a, l) in enumerate(curr_conversation):
        curr_a.extend(a)
        
        curr_l.extend(l)
        word_count += len(l)
        
        if cid == len(curr_conversation)-1 or word_count + len(curr_conversation[cid+1][0]) > max_seq_len:
            all_lines_short.append(f'{" ".join(curr_a)}\t{" ".join(curr_l)}')
            word_count = 0
            curr_a = []
            curr_l = []
        
with open('./train_data_short.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines_short))

In [ ]:
train_test_split = int(len(all_lines_short)*0.7)

with open('../bert-sequence-tagging/TEA/train_row__.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines_short[:train_test_split]))
    
with open('../bert-sequence-tagging/TEA/dev_row__.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_lines_short[train_test_split:]))